# 6주차 스타벅스 지도시각화 

## 1. URL 이동 및 추출할 사이트 찾기 

In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup

In [ ]:
browser = webdriver.Chrome('c:/webdriver/chromedriver.exe')

In [ ]:
url = 'https://www.istarbucks.co.kr/store/store_map.do'
browser.get(url)

In [ ]:
#지역검색 클릭
browser.find_element_by_css_selector('header.loca_search > h3 > a').click()

In [ ]:
# 서울 클릭 
browser.find_element_by_css_selector('ul.sido_arae_box > li:nth-child(1) > a').click()

In [ ]:
# 전체 클릭 
browser.find_element_by_css_selector('ul.gugun_arae_box > li:nth-child(1) > a').click()

## 2. HTML 파일 가져오기 

In [ ]:
# html 가져오기 
html = browser.page_source

In [ ]:
soup = BeautifulSoup(html, 'html.parser')

## 3. 매장 리스트 부분만 추출하기

In [ ]:
lst = soup.select('li.quickResultLstCon')
len(lst)

In [ ]:
lst = soup.select('ul.quickSearchResultBoxSidoGugun > li.quickResultLstCon')
len(lst)

In [ ]:
element = lst[0]
element

## 4. 정보 추출하기 
### 지점명 / 주소 / 전화번호 / 좌표정보(lat / long) / 매장유형 


* 지점명

In [ ]:
name_0 = element.select('strong')
name_0

In [ ]:
name_1 = name_0[0]
name_1

In [ ]:
name_2 = name_1.text
name_2

In [ ]:
name_3 = name_2.strip()
name_3

* 주소, 전화번호 찾기

In [ ]:
address_0 = element.select('p.result_details')
address_0

In [ ]:
address_1 = address_0[0]
address_1

In [ ]:
address_1.text

In [ ]:
address_2 = str(address_1).split('<br/>')
address_2

In [ ]:
address_3 = address_2[0]
address_3

In [ ]:
address_4 = address_3.split('>')
address_4

In [ ]:
address_5 = address_4[1]
address_5

In [ ]:
tel_0 = address_2[1]
tel_0

In [ ]:
tel_1 = tel_0.split('<')
tel_1

In [ ]:
tel_2 = tel_1[0]
tel_2

* 좌표정보

In [ ]:
element

In [ ]:
lat = element['data-lat']
lat

In [ ]:
long = element['data-long']
long

* 매장유형

In [ ]:
store_type_0 = element.select('i')
store_type_0

In [ ]:
store_type_1 = store_type_0[0]
store_type_1

In [ ]:
store_type_2 = store_type_1['class']
store_type_2

In [ ]:
store_type_3 = store_type_2[0]
store_type_3

### 정보 전체 정리 

In [ ]:
# 매장명 
store = element.select('strong')[0].text.strip()

# 주소 
address = str(element.select('p.result_details')[0]).split('<br/>')[0].split('>')[1]

# 전화번호 
tel = str(element.select('p.result_details')[0]).split('<br/>')[1].split('<')[0]

# 위도, 경도 
lat = element['data-lat']
long = element['data-long']

# 매장 타입 
store_type = element.select('i')[0]['class'][0]

print(store, address, tel, lat, long, store_type, sep = ' / ')

## 5. 반복문으로 전체 정보 추출하기 

In [ ]:
result = []

for element in lst: 
    # 매장명 
    store = element.select('strong')[0].text.strip()

    # 주소 
    address = str(element.select('p.result_details')[0]).split('<br/>')[0].split('>')[1]

    # 전화번호 
    tel = str(element.select('p.result_details')[0]).split('<br/>')[1].split('<')[0]

    # 위도, 경도 
    lat = element['data-lat']
    long = element['data-long']

    # 매장 타입 
    store_type = element.select('i')[0]['class'][0]
    
    element_result = [store, address, tel, lat, long, store_type]
    result.append(element_result)
    
result[0:5]

## 6. 데이터프레임으로 저장하기 

In [ ]:
import pandas as pd

In [ ]:
result_df = pd.DataFrame(result)
result_df.head()

In [ ]:
result_df.columns = ['매장명', '주소', '전화번호', '위도', '경도', '매장타입']
result_df

In [ ]:
result_df.to_excel('./starbucks_list_200312.xlsx', index = False)

## 7. 지도 시각화 하기 

In [ ]:
import folium 
import pandas as pd

In [ ]:
sb_lst = pd.read_excel('./starbucks_list_200312.xlsx')
sb_lst.head()

### 7.1. 지도에 점 찍어보기 

In [ ]:
# 지도의 레이어 표현하기 
sb_map = folium.Map(
    location = [37.573050, 126.979189], 
    tile = 'Stamen Terrain',
    zoom_start = 11
)
sb_map

In [ ]:
# 한개 추출하기 
sb_pt = sb_lst.head(1)
sb_pt

In [ ]:
sb_pt[['위도', '경도']]

In [ ]:
# 점 하나 찍어보기 

folium.CircleMarker(
    location = sb_pt[['위도', '경도']],
    fill = True, 
    fill_color = 'green', 
    fill_opacity = 1,
    color = 'yellow', 
    weight = 1,
    radius = 3
).add_to(sb_map)

sb_map

### 7.2. 반복문으로 스타벅스 매장 모두 지도에 표시해보기 

In [ ]:
# iterrows 함수 알아보기 
for x in sb_lst.iterrows():
    print( x)

In [ ]:
# 반복문으로 지도에 점 표기하기 
for i, row in sb_lst.iterrows(): 
    folium.CircleMarker(
        location = row[['위도', '경도']],
        fill = True, 
        fill_color = 'green', 
        fill_opacity = 1,
        color = 'yellow', 
        weight = 1,
        radius = 3
    ).add_to(sb_map)
    
sb_map

In [ ]:
# 스타벅스 매장 타입에 따라 다르게 나타내기 
sb_lst['매장타입'].unique()

In [ ]:
for i, row in sb_lst.iterrows():
    if row['매장타입'] == 'pin_general': 
        fill_color = 'green'
        line_color = 'yellow' 
    elif row['매장타입'] == 'pin_reserve': 
        fill_color = 'black'
        line_color = 'red' 
    elif row['매장타입'] == 'pin_generalDT': 
        fill_color = 'blue'
        line_color = 'black' 
        
    folium.CircleMarker(
        location = row[['위도', '경도']],
        fill = True, 
        fill_color = fill_color, 
        fill_opacity = 1,
        color = line_color, 
        weight = 1,
        radius = 3
    ).add_to(sb_map)
        
sb_map

### 7.3. folium 몇 가지 옵션 설정해보기 

In [ ]:
from matplotlib import font_manager, rc
import platform 

if platform.system() == 'Windows': 
    path = 'c:/Windows/Fonts/malgun.ttf'
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
elif platform.system() == 'Darwin':
    rc('font', family='AppleGothic')
else: 
    print('Check your OS system')


In [ ]:
# 팝업효과
for i, row in sb_lst.iterrows():
    if row['매장타입'] == 'pin_general': 
        fill_color = 'green'
        line_color = 'yellow' 
    elif row['매장타입'] == 'pin_reserve': 
        fill_color = 'black'
        line_color = 'red' 
    elif row['매장타입'] == 'pin_generalDT': 
        fill_color = 'blue'
        line_color = 'black' 
        
    popup = folium.Popup(html = row['매장명'], max_width=400)
    folium.CircleMarker(
        location = row[['위도', '경도']],
        fill = True, 
        fill_color = fill_color, 
        fill_opacity = 1,
        color = line_color, 
        weight = 1,
        radius = 3, 
        popup = popup
    ).add_to(sb_map)
        
sb_map

### 7.4. 지도 시각화 결과 저장하기 

In [ ]:
sb_map.save('starbucks_result.html')